In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#reading csvs
pop2020_df = pd.read_csv("pop2020.csv")
pop2019_df=pd.read_csv("pop2019.csv", encoding="ISO-8859-1")
house_df = pd.read_csv("All_house_price.csv")
inventory_df = pd.read_csv("Inventory_sales.csv")
rental_df = pd.read_csv("Rental_price.csv")

#cleaning popuation data sets
pop2020_df = pop2020_df.iloc[:, 1:4]
pop2020_df=pop2020_df.rename(columns={"Name" : "City",
                                      "2020 Pop":"2020 Population"})
pop2020_df = pop2020_df.replace({"New York City": "New York"})

colums = 'NAME', 'STNAME', 'POPESTIMATE2018', 'POPESTIMATE2019'
pop2019_df=pop2019_df.loc[:, colums]
pop2019_df=pop2019_df.rename(columns={"NAME" : "City",
                                      "STNAME" : "State",
                                      "POPESTIMATE2018": "2018 Population",
                                      "POPESTIMATE2019" : "2019 Population"})

#merging data
pop2020_df['City'] = pop2020_df['City'] + ' city'
population_df = pd.merge(pop2019_df, pop2020_df, on=["City", "State"], how='inner')
duplicated = population_df.loc[population_df.duplicated(subset=["City", "State"]), "City"]
pop_df = population_df.drop(duplicated.index)
pop_df.reset_index(drop=True, inplace=True)
pop_df["City"] = pop_df["City"].str.replace("city", "")

us_state_abbrev = {
'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}

pop_df['State'] = pop_df['State'].map(us_state_abbrev).fillna(pop_df['State'])

pop_df.to_csv("cleaned_population.csv")

#cleaning house price data
house_df = house_df.rename(columns={"RegionName":"City", "StateName":"State"})
house_df.drop(house_df.iloc[:, 5:271], inplace = True, axis = 1) 
house_df.drop(house_df.iloc[:, 0:2], inplace = True, axis = 1) 
del house_df["RegionType"]
house_df['City'] = house_df['City'].str.split(',').str[0]
house_df = house_df.replace({"Los Angeles-Long Beach-Anaheim": "Los Angeles",
                                "Dallas-Fort Worth":"Dallas",
                                 "Miami-Fort Lauderdale":"Miami",
                                 "Minneapolis-St Paul" : "Minneapolis",
                                 "Louisville-Jefferson County":"Louisville"})

In [88]:
#common cities in population data and house price data
Hcities = house_df[["City", "State"]]
Pcities = pop_df[["City", "State"]]
commons_df = pd.merge(Hcities, Pcities, on='City', how='right')
del commons_df["State_x"]
commons_df = commons_df.rename(columns={"State_y":"State"})

In [84]:
house_df = pd.merge(commons_df, house_df, on=["City", "State"], how='outer')
new_house_df=house_df.dropna(how="any")
new_house_df.reset_index(drop=True, inplace=True)
new_house_df.to_excel("housing.xls")
new_house_df

,City,State,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,...,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30
0,New York,NY,462565.0,465113.0,467189.0,469047.0,470161.0,471236.0,472394.0,473958.0,...,479999.0,480758.0,481745.0,482804.0,484104.0,485517.0,487279.0,489670.0,492875.0,497090.0
1,Los Angeles,CA,648303.0,652626.0,655982.0,659491.0,661457.0,661765.0,660802.0,661376.0,...,671796.0,675183.0,680320.0,685503.0,689705.0,691229.0,692332.0,696613.0,703740.0,711361.0
2,Chicago,IL,237993.0,238524.0,239302.0,240377.0,241087.0,241695.0,242195.0,242637.0,...,245373.0,245631.0,246017.0,246628.0,247155.0,247719.0,248421.0,249650.0,251298.0,253512.0
3,Dallas,TX,242582.0,244311.0,245753.0,247152.0,248345.0,249450.0,250150.0,251136.0,...,260294.0,260750.0,261428.0,262440.0,263584.0,264699.0,265992.0,267485.0,269183.0,270907.0
4,Philadelphia,PA,237000.0,238405.0,239485.0,240232.0,240937.0,241488.0,242160.0,242755.0,...,251872.0,252632.0,253576.0,254324.0,255509.0,256799.0,258666.0,260637.0,263126.0,265912.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,Lamesa,TX,67732.0,68037.0,68340.0,68591.0,68956.0,69279.0,69660.0,69953.0,...,75408.0,75568.0,75681.0,75786.0,75968.0,76213.0,76503.0,76738.0,76975.0,77216.0
909,Craig,CO,190370.0,189970.0,189519.0,188512.0,187773.0,187409.0,187368.0,187188.0,...,187489.0,187062.0,186934.0,186942.0,187037.0,187109.0,187262.0,187332.0,187393.0,187430.0
910,Pecos,TX,100363.0,101110.0,101969.0,103097.0,104205.0,105301.0,106710.0,108591.0,...,124046.0,125836.0,127175.0,128373.0,128879.0,129605.0,130574.0,131262.0,131311.0,130703.0
911,Vernon,TX,67665.0,67858.0,68007.0,68199.0,68269.0,68303.0,68435.0,68619.0,...,69702.0,69630.0,69495.0,69361.0,69277.0,69276.0,69229.0,69236.0,69248.0,69306.0


In [85]:
#inventory price to df
inventory_df = inventory_df.rename(columns={"RegionName":"City"})
inventory_df.drop(inventory_df.iloc[:, 3:125], inplace = True, axis = 1) 
inventory_df.drop(inventory_df.iloc[:, 0:2], inplace = True, axis = 1) 
inventory_df['City'] = inventory_df['City'].str.split(',').str[0]
inventory_df = inventory_df.replace({"Los Angeles-Long Beach-Anaheim": "Los Angeles",
                                "Dallas-Fort Worth":"Dallas",
                                 "Miami-Fort Lauderdale":"Miami",
                                 "Minneapolis-St Paul" : "Minneapolis",
                                 "Louisville-Jefferson County":"Louisville"})
inventory_df = pd.merge(pop_df["City"], inventory_df, on="City", how="outer")
new_inventory_df=inventory_df.dropna(how="any")
new_inventory_df.reset_index(drop=True, inplace=True)
new_inventory_df

,City,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,...,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31
0,United States,245307.0,245884.0,246400.0,246001.0,245551.0,244480.0,243086.0,246892.0,247803.0,...,258887.0,260206.0,261241.0,263097.0,264792.0,266846.0,267193.0,268139.0,270973.0,275416.0
1,New York,408581.0,410638.0,411282.0,412870.0,412813.0,412543.0,408396.0,413888.0,413266.0,...,424901.0,427462.0,428257.0,430321.0,432328.0,435443.0,436982.0,437493.0,437341.0,440403.0
2,Los Angeles,632317.0,636103.0,640302.0,639837.0,637670.0,633651.0,633124.0,639114.0,638244.0,...,651880.0,656392.0,662235.0,667281.0,671368.0,671098.0,671917.0,674008.0,680382.0,692580.0
3,Chicago,230644.0,229491.0,230750.0,230965.0,229788.0,228385.0,225892.0,230758.0,230267.0,...,238146.0,238984.0,239214.0,239167.0,241215.0,244731.0,247408.0,247870.0,248650.0,253130.0
4,Dallas,262498.0,264179.0,265362.0,266806.0,265729.0,263817.0,261708.0,263511.0,264086.0,...,274905.0,275471.0,276538.0,278098.0,279825.0,281356.0,280860.0,281504.0,283450.0,288052.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,Faribault,215361.0,206659.0,210820.0,209392.0,214077.0,216777.0,216415.0,220947.0,224170.0,...,245191.0,245679.0,243601.0,244028.0,248152.0,251663.0,255663.0,254834.0,259611.0,257119.0
117,Albemarle,146115.0,151291.0,146518.0,139950.0,142875.0,143897.0,152904.0,145765.0,149882.0,...,177721.0,181788.0,180336.0,182854.0,178686.0,177157.0,173216.0,174289.0,177662.0,177238.0
118,Kerrville,237722.0,242620.0,246300.0,250949.0,250204.0,247087.0,244873.0,249929.0,254266.0,...,265884.0,263247.0,258660.0,266176.0,273456.0,275588.0,274227.0,271993.0,277035.0,287515.0
119,Shelbyville,159953.0,161683.0,164469.0,162953.0,163395.0,168869.0,172399.0,171872.0,163950.0,...,190381.0,196688.0,197284.0,192164.0,188812.0,188837.0,195302.0,197259.0,200834.0,198129.0


In [86]:
#cleaning rental price data
rental_df = rental_df.rename(columns={"RegionName":"City"})
rental_df.drop(rental_df.iloc[:, 2:54], inplace = True, axis = 1) 
del rental_df ["RegionID"]
rental_df['City'] = rental_df['City'].str.split(',').str[0]
rental_df = rental_df.replace({"Los Angeles-Long Beach-Anaheim": "Los Angeles",
                                "Dallas-Fort Worth":"Dallas",
                                 "Miami-Fort Lauderdale":"Miami",
                                 "Minneapolis-St Paul" : "Minneapolis",
                                 "Louisville-Jefferson County":"Louisville"})

rental_df = pd.merge(pop_df["City"], rental_df, on="City", how="outer")
new_rental_df=rental_df.dropna(how="any")
new_rental_df.reset_index(drop=True, inplace=True)
new_rental_df

,City,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,...,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09
0,United States,1604.0,1609.0,1614.0,1619.0,1624.0,1629.0,1634.0,1639.0,1644.0,...,1698.0,1700.0,1702.0,1704.0,1706.0,1707.0,1709.0,1710.0,1711.0,1712.0
1,New York,2608.0,2612.0,2617.0,2621.0,2626.0,2631.0,2636.0,2642.0,2647.0,...,2698.0,2694.0,2686.0,2678.0,2671.0,2662.0,2653.0,2644.0,2635.0,2625.0
2,Los Angeles,2405.0,2414.0,2423.0,2432.0,2440.0,2448.0,2456.0,2464.0,2472.0,...,2542.0,2544.0,2545.0,2545.0,2546.0,2546.0,2546.0,2546.0,2545.0,2545.0
3,Chicago,1713.0,1716.0,1719.0,1722.0,1725.0,1728.0,1732.0,1735.0,1739.0,...,1778.0,1778.0,1777.0,1776.0,1776.0,1775.0,1774.0,1772.0,1771.0,1770.0
4,Dallas,1481.0,1485.0,1489.0,1493.0,1498.0,1502.0,1506.0,1511.0,1515.0,...,1560.0,1562.0,1563.0,1564.0,1566.0,1567.0,1568.0,1570.0,1571.0,1572.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,Port St. Lucie,1512.0,1520.0,1529.0,1538.0,1545.0,1552.0,1559.0,1563.0,1568.0,...,1640.0,1648.0,1656.0,1664.0,1671.0,1679.0,1687.0,1695.0,1702.0,1710.0
102,Fort Collins,1555.0,1559.0,1564.0,1569.0,1574.0,1579.0,1584.0,1589.0,1594.0,...,1636.0,1638.0,1641.0,1644.0,1646.0,1649.0,1651.0,1653.0,1656.0,1658.0
103,Boulder,1901.0,1906.0,1911.0,1916.0,1921.0,1926.0,1931.0,1936.0,1941.0,...,1991.0,1992.0,1993.0,1995.0,1996.0,1997.0,1998.0,1998.0,1999.0,2000.0
104,Greeley,1586.0,1592.0,1598.0,1604.0,1611.0,1617.0,1624.0,1629.0,1635.0,...,1680.0,1682.0,1685.0,1688.0,1690.0,1693.0,1695.0,1698.0,1700.0,1703.0
